In [1]:
from selenium import webdriver
driver=webdriver.Firefox()
driver.implicitly_wait(3)
driver.get("https://www.fundsexplorer.com.br/ranking")

In [2]:
from selenium.webdriver.common.by import By
fecha_aceitar = '//*[@id="hs-eu-confirmation-button"]'#"/html/body/div[2]/div/div[2]/div/a"
driver.find_element(By.XPATH,fecha_aceitar).click()

In [3]:
#-------------# Muda o contexto para o iframe 
iframe_element = driver.find_element(By.XPATH, "//iframe[@title='Popup CTA']")
driver.switch_to.frame(iframe_element)
# Encontra o botão "X" dentro do iframe
close_button = driver.find_element(By.XPATH, "/html/body/div/div[1]")
# Clique no botão "X"
close_button.click()
# Volte para o contexto principal
driver.switch_to.default_content()
#------------------------------------


In [4]:
# Scroll up the window by a specific number of pixels
# For example, scroll up by 200 pixels
scroll_distance = 500
driver.execute_script(f"window.scrollBy(0, {scroll_distance});")

In [5]:
driver.find_element(By.XPATH,'//*[@id="colunas-ranking__select-button"]').click()
driver.find_element(By.XPATH,'/html/body/div[7]/div[1]/div/div[2]/div[2]/ul/li[1]/label').click()

In [6]:
dados = []
dadosTabela = driver.find_element(By.XPATH,'//div/table[contains(@class,"default-fiis-table__container__table")]')

#print(dadosTabela.text)  

for linha in dadosTabela.find_elements(By.TAG_NAME,"tr") :
    linhaDados = []
    for coluna in linha.find_elements(By.TAG_NAME,"td"):
        # print(coluna)
        linhaDados.append(coluna.text)
    dados.append(linhaDados)


In [7]:
import pandas as pd
colunas = ['Fundos','Setor','Preço Atual (R$)','Liquidez Diária (R$)',#
    'P/VP','Último Dividendo','Dividend Yield','DY (3M) Acumulado',#
    'DY (6M) Acumulado','DY (12M) Acumulado','DY (3M) média','DY (6M) média',#
    'DY (12M) média','DY Ano','Variação Preço','Rentab. Período','Rentab. Acumulada',#
    'Patrimônio Líquido','VPA','P/VPA','DY Patrimonial','Variação Patrimonial',#
    'Rentab. Patr. Período','Rentab. Patr. Acumulada','Quant. Ativos' ,'Volatilidade' , #
    'Num. Cotistas' , 'Tax. Gestão' , 'Tax. Performance' , 'Tax. Administração']

df = pd.DataFrame(dados,columns=colunas)

df

,Fundos,Setor,Preço Atual (R$),Liquidez Diária (R$),P/VP,Último Dividendo,Dividend Yield,DY (3M) Acumulado,DY (6M) Acumulado,DY (12M) Acumulado,...,DY Patrimonial,Variação Patrimonial,Rentab. Patr. Período,Rentab. Patr. Acumulada,Quant. Ativos,Volatilidade,Num. Cotistas,Tax. Gestão,Tax. Performance,Tax. Administração
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,AAGR11,INDEFINIDO,"94,96","9.545,95",N/A,"1,37","1,43 %","3,99 %","8,11 %","15,72 %",...,"0,00 %","0,00 %","0,00 %","0,00 %",0,"67,24",0,N/A,N/A,N/A
2,AAZQ11,INDEFINIDO,"7,32","339.933,10","0,86","0,10","1,30 %","3,96 %","9,02 %","18,03 %",...,"1,10 %","0,00 %","0,00 %","0,00 %",0,"19,24",29.583,N/A,N/A,N/A
3,ABCP11,SHOPPINGS,"86,37","74.198,50","0,80","0,80","0,98 %","2,42 %","4,58 %","10,70 %",...,"0,56 %","17,08 %","17,73 %","26,11 %",1,"22,36",14.885,N/A,N/A,N/A
4,AFHI11,PAPÉIS,"92,84","1.027.525,35","0,98","1,01","1,11 %","3,30 %","6,64 %","13,17 %",...,"1,06 %","-0,26 %","0,79 %","7,11 %",14,"11,40",40.755,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,YUFI11B,INDEFINIDO,N/A,N/A,N/A,"0,63","0,00 %","0,00 %","0,00 %","0,00 %",...,"0,60 %","-0,01 %","0,59 %","6,32 %",0,N/A,0,N/A,N/A,N/A
530,ZAGH11,SERVIÇOS FINANCEIROS DIVERSOS,"10,34","933,10","1,11","0,07","0,69 %","2,07 %","3,75 %","7,14 %",...,N/A,N/A,N/A,N/A,1,"64,36",413,N/A,N/A,N/A
531,ZAVC11,INDEFINIDO,"8,86","5.860,90","0,90","0,13","0,00 %","3,06 %","7,24 %","15,43 %",...,N/A,N/A,N/A,N/A,1,"94,72",329,N/A,N/A,N/A
532,ZAVI11,INDEFINIDO,"91,05","53.721,25","0,70","1,18","1,29 %","3,72 %","7,62 %","14,65 %",...,N/A,N/A,N/A,N/A,9,"37,02",3.215,N/A,N/A,N/A


In [8]:
import os
data_path = str(os.getcwd()) + r"/data/"

In [9]:
df.to_csv( data_path+"FE.csv" , sep=";" , decimal="," )

In [10]:
driver.close()